# Interpretability Techniques for single samples from FUNSD

## Github

In [ ]:
!git clone https://github.com/adamserag1/Interpretability-for-VRDU-models.git

In [ ]:
!git pull https://github.com/adamserag1/Interpretability-for-VRDU-models.git

In [ ]:
%cd /content/Interpretability-for-VRDU-models

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install -U datasets

## Libraries

In [ ]:
#code
from datasets import load_from_disk
from transformers import LayoutLMv3ForSequenceClassification, AutoProcessor, BrosModel, AutoTokenizer, BrosPreTrainedModel, AutoConfig
import sys
import importlib
def reload_modules():
    for module in list(sys.modules.keys()):
        if module.startswith('vrdu_utils') or module.startswith('Classification_Explain') or module.startswith('lime') or module.startswith('Eval'):
            print(f"Reloading module: {module}")
            importlib.reload(sys.modules[module])

reload_modules()

from vrdu_utils.encoders import *
from Classification_Explain.lime import *
from vrdu_utils.utils import *
import torch
from Eval.eval_suite import *
from Eval.fidelity import *
from Classification_Explain.shap import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import warnings
from transformers import logging as hf_logging

warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    module="transformers.modeling_utils",   # the module that emits the msg
)
hf_logging.set_verbosity_error()


## Data & Model Setup

In [ ]:
FUNSD = load_dataset('nielsr/funsd')
print(FUNSD)